In [1]:
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use('dark_background')

In [75]:
df = pd.read_csv('data/train.csv', index_col=0)
val_df = pd.read_csv('data/test.csv', index_col=0)

df = pd.concat([df, pd.get_dummies(data=df.MSZoning, prefix='MSZoning')], axis=1)
df.LotFrontage.fillna(df.LotFrontage.mean(), inplace=True)
df = pd.concat([df, pd.get_dummies(data=df.Street, prefix='Street')], axis=1)
df.Alley.fillna('Not precised', inplace=True)
df = pd.concat([df, pd.get_dummies(data=df.Alley, prefix='Alley')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.LotShape, prefix='LotShape')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.LandContour, prefix='LandContour')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.Utilities, prefix='Utilities')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.LotConfig, prefix='LotConfig')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.LandSlope, prefix='LandSlope')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.Condition1, prefix='Condition1')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.Condition2, prefix='Condition2')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.Neighborhood, prefix='Neighborhood')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.BldgType, prefix='BldgType')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.HouseStyle, prefix='HouseStyle')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.RoofStyle, prefix='RoofStyle')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.RoofMatl, prefix='RoofMatl')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.Exterior1st, prefix='Exterior1st')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.Exterior2nd, prefix='Exterior2nd')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.MasVnrType, prefix='MasVnrType')], axis=1)
df.MasVnrArea.fillna('None', inplace=True)
df = pd.concat([df, pd.get_dummies(data=df.MasVnrArea, prefix='MasVnrArea')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.ExterQual, prefix='ExterQual')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.ExterCond, prefix='ExterCond')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.Foundation, prefix='Foundation')], axis=1)
df.BsmtQual.fillna('Not precised', inplace=True)
df.BsmtCond.fillna('Not precised', inplace=True)
df.BsmtExposure.fillna('Not precised', inplace=True)
df = pd.concat([df, pd.get_dummies(data=df.BsmtQual, prefix='BsmtQual')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.BsmtCond, prefix='BsmtCond')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.BsmtExposure, prefix='BsmtExposure')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.BsmtFinType1, prefix='BsmtFinType1')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.BsmtFinType2, prefix='BsmtFinType2')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.Heating, prefix='Heating')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.HeatingQC, prefix='HeatingQC')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.CentralAir, prefix='CentralAir')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.Electrical, prefix='Electrical')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.KitchenQual, prefix='KitchenQual')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.Functional, prefix='Functional')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.FireplaceQu, prefix='FireplaceQu')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.GarageType, prefix='GarageType')], axis=1)
# tutaj nie wiem czy podzielic na kolumny po wartosciach liczbowych czy jest lepszy sposob
df.GarageYrBlt.fillna('Not built', inplace=True)
df = pd.concat([df, pd.get_dummies(data=df.GarageYrBlt, prefix='GarageYrBlt')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.GarageFinish, prefix='GarageFinish')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.GarageQual, prefix='GarageQual')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.GarageCond, prefix='GarageCond')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.PavedDrive, prefix='PavedDrive')], axis=1)
df.PoolQC.fillna('None', inplace=True)
df.Fence.fillna('None', inplace=True)
df = pd.concat([df, pd.get_dummies(data=df.PoolQC, prefix='PoolQC')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.Fence, prefix='Fence')], axis=1)
df.MiscFeature.fillna('None', inplace=True)
df = pd.concat([df, pd.get_dummies(data=df.MiscFeature, prefix='MiscFeature')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.SaleType, prefix='SaleType')], axis=1)
df = pd.concat([df, pd.get_dummies(data=df.SaleCondition, prefix='SaleCondition')], axis=1)

df.drop(columns=['Street', 'MSZoning', 'Alley', 'LotShape', 'LandContour', 'Unique', 'Utilities', 'LotConfig',
                 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle',
                 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
                 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC',
                 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish',
                 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'
                 ], inplace=True, errors='ignore')

In [76]:
df

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
1,60,65.0,8450,7,5,2003,2003,196.0,706,0,...,0,0,0,1,0,0,0,0,1,0
2,20,80.0,9600,6,8,1976,1976,0.0,978,0,...,0,0,0,1,0,0,0,0,1,0
3,60,68.0,11250,7,5,2001,2002,162.0,486,0,...,0,0,0,1,0,0,0,0,1,0
4,70,60.0,9550,7,5,1915,1970,0.0,216,0,...,0,0,0,1,1,0,0,0,0,0
5,60,84.0,14260,8,5,2000,2000,350.0,655,0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,60,62.0,7917,6,5,1999,2000,0.0,0,0,...,0,0,0,1,0,0,0,0,1,0
1457,20,85.0,13175,6,6,1978,1988,119.0,790,163,...,0,0,0,1,0,0,0,0,1,0
1458,70,66.0,9042,7,9,1941,2006,0.0,275,0,...,0,0,0,1,0,0,0,0,1,0


In [77]:
# df.isna().any()
df.isnull().values.any()

False